# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

#configure map
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv and DF
vacationDF = pd.read_csv("weatherdata.csv")
vacationDF.drop(vacationDF.columns[vacationDF.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
vacationDF.head()

,City,Latitude,Longitude,Temperature,Cloudiness,Wind Speed,Humidity,Country,Date
0,Cidreira,-30.18,-50.21,71.98,13,10.89,72,BR,1603913929
1,Paarl,-33.73,18.98,59.29,75,19.46,82,ZA,1603913799
2,Mar del Plata,-38.00,-57.56,62.01,90,5.82,59,AR,1603913748
3,Saint-Augustin,51.23,-58.65,31.42,19,6.51,82,CA,1603913771
4,Malanje,-9.54,16.34,67.17,100,5.88,95,AO,1603914005


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#set weight and locations
humidity = vacationDF["Humidity"]
locations = vacationDF[["Latitude", "Longitude"]]

#put onto gmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=5,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
#Narrow down to wind speed less than 10 mph, no cloudiness and a temperature more than 70 F but less than 80F
newVacation = vacationDF.loc[(vacationDF["Wind Speed"]<10)&(vacationDF["Cloudiness"]==0)&(vacationDF["Temperature"]>70)&(vacationDF["Temperature"]<80)].dropna()
newVacation

,City,Latitude,Longitude,Temperature,Cloudiness,Wind Speed,Humidity,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#new DF
hotel_df = pd.DataFrame(newVacation, columns=["City", "Country", "Latitude", "Longitude"])


#add column
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name


In [21]:
#loop to search for hotels within 5000 meters
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    long = row["Longitude"]
    name = row["City"]

    params["location"] = f"{lat},{long}"

    print(f"Retrieving Results for Index {index}: {name}.")
    response = requests.get(url, params=params).json()

    results = response['results']

    try:
        print(f"Closest hotel in {name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field... skipping.")
        
    print("------------")

In [ ]:
#display the available hotels
hotel_df

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))